<a href="https://colab.research.google.com/github/PavelStelmakhV/hw309-selection-of-hyperparameters/blob/main/hw309.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [46]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from keras import layers
from keras import callbacks
from keras import initializers
from keras import regularizers
from keras import optimizers

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import BatchNormalization

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
from tensorboard.plugins.hparams import api as hp

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical


# import numpy as np
# import matplotlib.pyplot as plt
# import tf_keras

# from keras import layers
# from keras import models
# from keras import regularizers
# from keras import callbacks
# from keras import initializers


# from keras.models import load_model


In [47]:
# !rm -rf ./logs/

In [48]:
num_classes = 10
log_dir = 'logs/hparam_tuning/'

In [49]:
%%capture
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [50]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

In [51]:
PATIENCE = 6
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([256, 512]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.25, 0.5))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.3])) # [0.1, 0.2, 0.3, 0.4, 0.5] 0.33
HP_DROPOUT_2 = hp.HParam('dropout_2', hp.Discrete([0.3])) # [0.1, 0.2, 0.3, 0.4, 0.5] 0.25
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam'])) # ['sgd', 'rmsprop', 'adam', 'nadam']
HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([128, 256, 512]))
HP_NUM_LAYERS = hp.HParam('num_layers', hp.Discrete([2, 3]))
HP_L2 = hp.HParam('l_2', hp.Discrete([ 5e-6, 1e-6, 5e-7])) #   6e-7                 [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7]
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([8e-4, 1e-3, 2e-3])) #  8e-4      [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7]
HP_ACTIVATION = hp.HParam('activation', hp.Discrete(['relu'])) # ['relu', 'tanh', 'sigmoid', 'elu']

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER, HP_BATCH_SIZE, HP_NUM_LAYERS, HP_L2, HP_DROPOUT_2, HP_ACTIVATION, HP_LEARNING_RATE],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [52]:
def deep_layer(model, neurons, activation, drop_out):
  w_init_relu = initializers.HeNormal(seed=42)
  b_init = initializers.Zeros()

  model.add(BatchNormalization())
  model.add(layers.Dense(neurons,
                        activation=activation,
                        kernel_initializer=w_init_relu,
                        bias_initializer=b_init,
                        kernel_regularizer=regularizers.L2(l2=hparams[HP_L2]),
                        # # kernel_regularizer=regularizers.L1L2(l1=l1, l2=l2),
                        bias_regularizer=regularizers.L2(l2=hparams[HP_L2]),
                        # # bias_regularizer=regularizers.L1L2(l1=l1, l2=l2),
                        activity_regularizer=regularizers.L2(l2=hparams[HP_L2])
                        # # activity_regularizer=regularizers.L1L2(l1=l1, l2=l2)
                         )
  )
  model.add(layers.Dropout(drop_out))
  return model

In [53]:
def train_test_model(run_dir, hparams):

    model = Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    model.add(layers.Dropout(hparams[HP_DROPOUT_2]))

    # model = deep_layer(model, hparams[HP_NUM_UNITS] / 2, hparams[HP_ACTIVATION], hparams[HP_DROPOUT])
    for num_layer in range(hparams[HP_NUM_LAYERS]):
      model = deep_layer(model, hparams[HP_NUM_UNITS], hparams[HP_ACTIVATION], hparams[HP_DROPOUT])
      # print(num_layer, hparams[HP_NUM_LAYERS])
    model.add(layers.Dense(10, activation='softmax'))

    optimizer = optimizers.Nadam(learning_rate=hparams[HP_LEARNING_RATE])

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy'],
    )

    callback_list = [
        EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=PATIENCE),
        ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True),
        TensorBoard(log_dir=run_dir, histogram_freq=1)
    ]
    # Train the model
    model.fit(x=x_train,
              y=y_train,
              epochs=200,
              batch_size=hparams[HP_BATCH_SIZE],
              validation_split=0.13,
              callbacks=callback_list,
              verbose=0
              )

    # model.summary()

    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(x_test, y_test)
    print(f"Test Accuracy: {test_accuracy}")
    return test_accuracy


In [54]:
def run(run_dir, hparams, step):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(run_dir, hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=step)

In [ ]:
session_num = 10
for num_units in HP_NUM_UNITS.domain.values:
  # for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
  for dropout_rate in HP_DROPOUT.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:
      for batch_size in HP_BATCH_SIZE.domain.values:
        for num_layers in HP_NUM_LAYERS.domain.values:
          for l_2 in HP_L2.domain.values:
            for dropout_rate_2 in HP_DROPOUT_2.domain.values:
              for activation in HP_ACTIVATION.domain.values:
                for lerning_rate in HP_LEARNING_RATE.domain.values:
                  hparams = {
                      HP_NUM_UNITS: num_units,
                      HP_DROPOUT: dropout_rate,
                      HP_OPTIMIZER: optimizer,
                      HP_BATCH_SIZE: batch_size,
                      HP_NUM_LAYERS: num_layers,
                      HP_L2: l_2,
                      HP_DROPOUT_2: dropout_rate_2,
                      HP_ACTIVATION: activation,
                      HP_LEARNING_RATE: lerning_rate,
                  }
                  run_name = "run-%d" % session_num
                  run_name = run_name + f" lr[{lerning_rate}] a[{activation}] [{optimizer}] d[{dropout_rate}] d2[{dropout_rate_2}] n[{num_units}] bs[{batch_size}] nl[{num_layers}] [{l_2}] [{PATIENCE}]0.09"
                  print('--- Starting trial: %s' % run_name)
                  print({h.name: hparams[h] for h in hparams})

                  run(log_dir + run_name, hparams, session_num)
                  session_num += 1

Выходные данные были обрезаны до нескольких последних строк (5000).

Epoch 15: val_accuracy improved from 0.88462 to 0.88808, saving model to best_model.h5

Epoch 16: val_accuracy did not improve from 0.88808

Epoch 17: val_accuracy did not improve from 0.88808

Epoch 18: val_accuracy did not improve from 0.88808

Epoch 19: val_accuracy did not improve from 0.88808

Epoch 20: val_accuracy improved from 0.88808 to 0.89051, saving model to best_model.h5

Epoch 21: val_accuracy improved from 0.89051 to 0.89333, saving model to best_model.h5

Epoch 22: val_accuracy did not improve from 0.89333

Epoch 23: val_accuracy did not improve from 0.89333

Epoch 24: val_accuracy did not improve from 0.89333

Epoch 25: val_accuracy did not improve from 0.89333

Epoch 26: val_accuracy did not improve from 0.89333

Epoch 27: val_accuracy did not improve from 0.89333

Epoch 28: val_accuracy improved from 0.89333 to 0.89513, saving model to best_model.h5

Epoch 29: val_accuracy did not improve from 0.895

In [ ]:
%tensorboard --logdir logs/hparam_tuning